In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# Get Item Information from Search Results

For each search result page, scrape the general information such as board board name, the number of reviews, the average rating, and the price.

In [309]:
def get_search_results_items(no_pages):
    headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}
    
    alls = []
    # scrape i number of pages
    for i in range(1, no_pages+1):
        r = requests.get('https://www.amazon.com/s?k=board+games&page='+str(i), headers=headers)
        content = r.content
        soup = BeautifulSoup(content)

        # for each board game...scrape the asin(amazon product #), name, price, rating, # of ratings
        for d in soup.findAll('div', attrs={'class':'sg-col-4-of-24 sg-col-4-of-12 sg-col-4-of-36 s-result-item s-asin sg-col-4-of-28 sg-col-4-of-16 sg-col sg-col-4-of-20 sg-col-4-of-32'}):
            name = d.find('span', attrs={'class':'a-size-base-plus a-color-base a-text-normal'})
            price = d.find('span', attrs={'class':'a-offscreen'})
            rating = d.find('span', attrs={'class':'a-icon-alt'})
            no_of_ratings = d.find('span', attrs={'class':'a-size-base'})
            asin = d['data-asin']

            # append information into a list
            all1=[]
            
            if asin is not None:
                all1.append(asin)
            else:
                all1.append('-')
            
            if name is not None:
                all1.append(name.text)
            else:
                all1.append("unknown-product")

            if price is not None:
                all1.append(price.text)
            else:
                all1.append('$0')

            if rating is not None:
                all1.append(rating.text)
            else:
                all1.append('-')
                
            if no_of_ratings is not None:
                all1.append(no_of_ratings.text)
            else:
                all1.append('-') 
                

            # for each item, append all1 list into alls list
            alls.append(all1)  

    return alls

In [310]:
# convert list of lists into dataframe
items_df = pd.DataFrame(get_search_results_items(10),columns=['asin_id',
                                       'name',
                                       'price',
                                       'avg_rating',
                                       'no_of_ratings'])

In [311]:
items_df.to_csv('csv_files/items_df.csv')

## Items - Preprocessing

In [313]:
items_df

,asin_id,name,price,avg_rating,no_of_ratings
0,B076HK9H7Z,Sorry! Game,$0,4.7 out of 5 stars,"7,555"
1,B08GD242TJ,Hasbro Gaming Guess Who? Game Original Guessin...,$9.97,4.9 out of 5 stars,30
2,B00C0ULS3G,Battleship Classic Board Game Strategy Game Ag...,$0,4.7 out of 5 stars,"4,151"
3,B08636S37Z,Funko Disney The Haunted Mansion – Call of The...,$24.97,4.7 out of 5 stars,84
4,B07B7KS87Q,Game Of Life,$19.99,4.7 out of 5 stars,"4,284"
...,...,...,...,...,...
301,B08GG9J8HR,Coatl - Strategy Board Game for 1-4 Players,$39.99,4.8 out of 5 stars,7
302,B07M5BMVS3,CLUE: Scooby Doo! Board Game | Official Scooby...,$39.95,4.7 out of 5 stars,341
303,B07HM7WZT9,Rio Grande Games Power Grid Recharged,$35.70,4.8 out of 5 stars,132
304,B083Q2FKTN,Escape Room The Game Expansion Pack – The Brea...,$11.99,5.0 out of 5 stars,1


### Remove duplicates

In [314]:
items_df = items_df.drop_duplicates(subset=['asin_id'])

### Convert price to float values

In [315]:
# convert price column as float values

# strip '?' symbol from price
items_df['price'] = items_df['price'].apply(lambda x:x.strip('$'))

# convert price column to float
items_df['price'] = items_df['price'].astype('float')

# check if column was converted to float values
items_df['price']

/Users/thomasttam/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/Users/thomasttam/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


0       0.00
1       9.97
2       0.00
3      24.97
4      19.99
       ...  
300    14.95
301    39.99
302    39.95
303    35.70
304    11.99
Name: price, Length: 252, dtype: float64

### Clean rating column by removing unnecessary characters

In [316]:
items_df['avg_rating'].value_counts()

4.8 out of 5 stars    68
4.7 out of 5 stars    51
4.6 out of 5 stars    32
-                     21
4.5 out of 5 stars    19
4.4 out of 5 stars    18
5.0 out of 5 stars    14
4.9 out of 5 stars    12
4.3 out of 5 stars     8
4.2 out of 5 stars     3
4.1 out of 5 stars     3
4.0 out of 5 stars     1
3.8 out of 5 stars     1
3.9 out of 5 stars     1
Name: avg_rating, dtype: int64

In [317]:
# remove items with '-' rating
items_df = items_df[items_df['avg_rating'] != '-'].reset_index(drop=True)

In [318]:
# get the number rating from the rating column by taking the left characters of string
items_df['avg_rating'] = items_df['avg_rating'].apply(lambda x: x[0:3])
# convert column to float vlaues
items_df['avg_rating'] = items_df['avg_rating'].astype('float')

### Convert no_of_rating into int values

In [319]:
# replace commas with empty space
items_df['no_of_ratings'] = items_df['no_of_ratings'].apply(lambda x: x.replace(',',''))

# # strip characters with empty spaces such as ' by '
items_df['no_of_ratings'] = items_df['no_of_ratings'].apply(lambda x: x.strip())

# # once stripped, remove items with '-' rating and 'by' rating
items_df = items_df[(items_df['no_of_ratings'] != '-') & (items_df['no_of_ratings'] != 'by')].reset_index(drop=True)

# # cast column as float values
items_df['no_of_ratings'] = items_df['no_of_ratings'].astype('int')

In [320]:
items_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 228 entries, 0 to 227
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   asin_id        228 non-null    object 
 1   name           228 non-null    object 
 2   price          228 non-null    float64
 3   avg_rating     228 non-null    float64
 4   no_of_ratings  228 non-null    int64  
dtypes: float64(2), int64(1), object(2)
memory usage: 9.0+ KB


In [321]:
items_df

,asin_id,name,price,avg_rating,no_of_ratings
0,B076HK9H7Z,Sorry! Game,0.00,4.7,7555
1,B08GD242TJ,Hasbro Gaming Guess Who? Game Original Guessin...,9.97,4.9,30
2,B00C0ULS3G,Battleship Classic Board Game Strategy Game Ag...,0.00,4.7,4151
3,B08636S37Z,Funko Disney The Haunted Mansion – Call of The...,24.97,4.7,84
4,B07B7KS87Q,Game Of Life,19.99,4.7,4284
...,...,...,...,...,...
223,B01C5U4PTY,"Rubik's Race Game, Head To Head Fast Paced Squ...",14.95,4.5,3981
224,B08GG9J8HR,Coatl - Strategy Board Game for 1-4 Players,39.99,4.8,7
225,B07M5BMVS3,CLUE: Scooby Doo! Board Game | Official Scooby...,39.95,4.7,341
226,B07HM7WZT9,Rio Grande Games Power Grid Recharged,35.70,4.8,132


### save processed items_df as csv

In [322]:
# save as csv file
items_df.to_csv('csv_files_test/cleaned_items_df.csv')

# Get Reviews for each Item

For each asin id, scrape reviews from the first 2 review pages.

In [323]:
def get_reviews(asin_no, no_review_pages):
    headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}
    
    alls = []
    # for each asin, scrape i number of pages
    for i in range(1, no_review_pages+1):
        r = requests.get('https://www.amazon.com/product-reviews/'+asin_no+'/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber='+str(i), headers=headers)
        content = r.content
        soup = BeautifulSoup(content)

        # scrape the following for each page
        for d in soup.findAll('div', attrs={'class':'a-section review aok-relative'}):
            product_id = d['id']
            review_title = d.find('a', attrs={'class':'a-size-base a-link-normal review-title a-color-base review-title-content a-text-bold'})
            review_description = d.find('span', attrs={'class':'a-size-base review-text review-text-content'})
            rating = d.find('span', attrs={'class':'a-icon-alt'})
#             user_name = d.find('span', attrs={'class': 'a-profile-name'}).text

            all1=[]
            
            all1.append(asin_no)
            
            if product_id is not None:
                all1.append(product_id)
            else:
                all1.append("unknown-product")

            if review_title is not None:
                all1.append(review_title.text)
            else:
                all1.append('$0')

            if review_description is not None:
                #print(rating.text)
                all1.append(review_description.text)
            else:
                all1.append('-1')

            if rating is not None:
                #print(price.text)
                all1.append(rating.text)
            else:
                all1.append('0')


            alls.append(all1)  

    return alls

## Scrape on several occasions using different VPN to prevent slamming the server (October 13, 2020)

In [325]:
# scraper 1
# append reviews from all items to review list

reviews = []

for id_no in items_df['asin_id'][0:100]:
    for row in get_reviews(id_no,3):
        reviews.append(row)

In [327]:
# scraper 2
# append reviews from all items to review list

reviews1 = []

for id_no in items_df['asin_id'][101:150]:
    for row in get_reviews(id_no,3):
        reviews1.append(row)

In [329]:
# scraper 3
# append reviews from all items to review list

reviews2 = []

for id_no in items_df['asin_id'][151:200]:
    for row in get_reviews(id_no,3):
        reviews2.append(row)

In [331]:
# scraper 4
# append reviews from all items to review list

reviews3 = []

for id_no in items_df['asin_id'][201:250]:
    for row in get_reviews(id_no,3):
        reviews3.append(row)

In [333]:
# scraper 5
# append reviews from all items to review list

reviews4 = []

for id_no in items_df['asin_id'][251:304]:
    for row in get_reviews(id_no,3):
        reviews4.append(row)

### Combine all reviews into Dataframe

In [335]:
reviews_df = pd.DataFrame(reviews + reviews1 + reviews2 + reviews3 + reviews4,columns=['asin_id',
                                                                                       'review_id',
                                                                                       'review_title',
                                                                                       'review_description',
                                                                                       'user_rating'])

In [336]:
reviews_df.to_csv('csv_files/reviews_df.csv')

## Preprocessing - Reviews

In [338]:
reviews_df.head()

,asin_id,review_id,review_title,review_description,user_rating
0,B076HK9H7Z,R1OSPWS88F2CUZ,DO NOT BUY!!!,I would give this zero stars if I could! If ...,1.0
1,B076HK9H7Z,R1DCFJ8VYSN17B,Is this the millennial version?,This is not the original sorry game. It only...,1.0
2,B076HK9H7Z,R1V07N4GXA7RSL,Wimp and Crybaby Edition,We bought this to replace our old Sorry game...,1.0
3,B076HK9H7Z,R2Z262NZDEU2EY,NOT the original/regular Sorry!,Be warned that this is not the sorry you gre...,2.0
4,B076HK9H7Z,RG3XIFV1PUX9Y,"Not the classic by a long shot, but okay.","Definitely not the classic game, with only 3...",4.0


In [339]:
reviews_df.shape

(6693, 5)

### scraping didn't lead to great results - price was in review_title so will remove them (186 rows)

In [340]:
reviews_df = reviews_df[reviews_df['review_title'] != '$0']
reviews_df.head()

,asin_id,review_id,review_title,review_description,user_rating
0,B076HK9H7Z,R1OSPWS88F2CUZ,DO NOT BUY!!!,I would give this zero stars if I could! If ...,1.0
1,B076HK9H7Z,R1DCFJ8VYSN17B,Is this the millennial version?,This is not the original sorry game. It only...,1.0
2,B076HK9H7Z,R1V07N4GXA7RSL,Wimp and Crybaby Edition,We bought this to replace our old Sorry game...,1.0
3,B076HK9H7Z,R2Z262NZDEU2EY,NOT the original/regular Sorry!,Be warned that this is not the sorry you gre...,2.0
4,B076HK9H7Z,RG3XIFV1PUX9Y,"Not the classic by a long shot, but okay.","Definitely not the classic game, with only 3...",4.0


### Parse user rating and convert to float values

In [341]:
reviews_df['user_rating'].value_counts()

5.0    4301
4.0     887
1.0     657
3.0     517
2.0     331
Name: user_rating, dtype: int64

In [342]:
# # get the number rating from the review_rating column by taking the left characters of string
# reviews_df['user_rating'] = reviews_df['user_rating'].apply(lambda x: x[0:3])
# # convert column to float vlaues
# reviews_df['user_rating'] = reviews_df['user_rating'].astype('float')
# reviews_df

### Clean the '/n' from review_title and review_description

In [343]:
# strip '/n' characters from title and description
reviews_df['review_title'] = reviews_df['review_title'].apply(lambda x:x.strip('\n'))

# was getting an error when stripping review_description due to N/A (row 56)
reviews_df['review_description'][reviews_df['review_description'].isna()]
reviews_df['review_description'] = reviews_df['review_description'].fillna('-1')
reviews_df['review_description'] = reviews_df['review_description'].apply(lambda x:x.strip('\n'))

In [344]:
reviews_df.head()

,asin_id,review_id,review_title,review_description,user_rating
0,B076HK9H7Z,R1OSPWS88F2CUZ,DO NOT BUY!!!,I would give this zero stars if I could! If ...,1.0
1,B076HK9H7Z,R1DCFJ8VYSN17B,Is this the millennial version?,This is not the original sorry game. It only...,1.0
2,B076HK9H7Z,R1V07N4GXA7RSL,Wimp and Crybaby Edition,We bought this to replace our old Sorry game...,1.0
3,B076HK9H7Z,R2Z262NZDEU2EY,NOT the original/regular Sorry!,Be warned that this is not the sorry you gre...,2.0
4,B076HK9H7Z,RG3XIFV1PUX9Y,"Not the classic by a long shot, but okay.","Definitely not the classic game, with only 3...",4.0


In [345]:
reviews_df.shape

(6693, 5)

### Remove duplicate reviews

In [346]:
reviews_df = reviews_df.drop_duplicates(subset=['review_id'])

In [347]:
reviews_df.shape

(5539, 5)

### save processed reviews_df as csv

In [348]:
reviews_df.to_csv('csv_files/cleaned_reviews_df.csv')